In [1]:
random_seeds = [42, 100, 0, 10, 12, 20, 50, 9, 30, 51]
run = 0

In [2]:
username = 'meganorm-mznasrabadi'
datasets = {
    'BTNRH': {
        'base_dir': '/project/meganorm/Data/BTNRH/BIDS',
        'task': "task-rest",
        "ending" : "meg.fif"
        },
    'CAMCAN': {
        'base_dir': '/project/meganorm/Data/camcan/BIDS',
        'task': "task-rest",
        "ending" : "meg.fif"
        },
    'NIMH': {
        'base_dir': '/project/meganorm/Data/NIMH',
        'task': "task-rest",
        "ending" : "meg.ds"
        },
    'OMEGA': {
        'base_dir': '/project/meganorm/Data/Omega',
        'task': "task-rest",
        "ending" : "meg.ds"
        },
    'HCP': {
        'base_dir': '/project/meganorm/Data/HCP',
        'task': "",
        "ending" : "4-Restin/4D"
        },
    'MOUS': {
        'base_dir': '/project/meganorm/Data/MOUS',
        'task': "task-rest",
        "ending" : "meg.ds"
    }
    }

package_path = f'/home/{username}/MEGaNorm/'

In [3]:
import os
os.chdir(package_path)
from utils.parallel import submit_jobs, check_jobs_status, collect_results
from utils.nm import hbr_data_split, estimate_centiles, evaluate_mace, shapiro_stat, abnormal_probability
from plots.plots import plot_nm_range_site2, plot_comparison, plot_neurooscillochart, plot_age_dist2, plot_growthcharts, plot_quantile_gauge, box_plot_auc, z_scores_scatter_plot, plot_metrics
from utils.nm import model_quantile_evaluation, calculate_oscilochart, prepare_prediction_data, cal_stats_for_gauge, aggregate_metrics_across_runs
from utils.IO import merge_datasets_with_regex, merge_fidp_demo, merge_datasets_with_glob
import pandas as pd
import json
from pcntoolkit.normative_parallel import execute_nm, rerun_nm, collect_nm
import warnings
import pickle
import numpy as np
from pcntoolkit.util.utils import z_to_abnormal_p, anomaly_detection_auc
from scipy.stats import false_discovery_control
warnings.filterwarnings("ignore")

# Configuration

In [4]:
def make_config(project, path=None):

    # preprocess configurations =================================================
    # downsample data
    config = dict()

    # You could also set layout to None to have high 
    # choices: all, lobe, None
    config["which_layout"] = "all"

    # which sensor type should be used
    # choices: meg, mag, grad, eeg, opm
    config["which_sensor"] = "meg"
    # config['fs'] = 1000

    # ICA configuration
    config['ica_n_component'] = 30
    config['ica_max_iter'] = 800
    config['ica_method'] = "fastica"

    # lower and upper cutoff frequencies in a bandpass filter
    config['cutoffFreqLow'] = 1
    config['cutoffFreqHigh'] = 45

    config["resampling_rate"] = 1000
    config["digital_filter"] = True
    config["notch_filter"] = False

    config["apply_ica"] = True

    config["auto_ica_corr_thr"] = 0.9

    # options are "average", "REST", and None 
    config["rereference_method"]= "average"

    # variance threshold across time
    config["mag_var_threshold"] = 4e-12
    config["grad_var_threshold"] = 4000e-13
    config["eeg_var_threshold"] = 40e-6
    # flatness threshold across time
    config["mag_flat_threshold"] = 10e-15
    config["grad_flat_threshold"] = 10e-15
    config["eeg_flat_threshold"] = 40e-6
    # variance thershold across channels
    config["zscore_std_thresh"] = 15 # change this

    # segmentation ==============================================
    #start time of the raw data to use in seconds, this is to avoid possible eye blinks in close-eyed resting state. 
    config['segments_tmin'] = 20
    # end time of the raw data to use in seconds, this is to avoid possible eye blinks in close-eyed resting state.
    config['segments_tmax'] = -20
    # length of MEG segments in seconds
    config['segments_length'] = 10
    # amount of overlap between MEG sigals in seconds
    config['segments_overlap'] = 2

    # PSD ==============================================
    # Spectral estimation method
    config['psd_method'] = "welch"
    # amount of overlap between windows in Welch's method
    config['psd_n_overlap'] = 1
    config['psd_n_fft'] = 2
    # number of samples in psd
    config["psd_n_per_seg"] = 2

    # fooof analysis configurations ==============================================
    # Desired frequency range to run FOOOF
    config['fooof_freqRangeLow'] = 3
    config['fooof_freqRangeHigh'] = 40
    # which mode should be used for fitting; choices (knee, fixed)
    config["aperiodic_mode"] = "knee"
    # minimum acceptable peak width in fooof analysis
    config["fooof_peak_width_limits"] = [1.0, 12.0]
    #Absolute threshold for detecting peaks
    config['fooof_min_peak_height'] = 0
    #Relative threshold for detecting peaks
    config['fooof_peak_threshold'] = 2

    # feature extraction ==========================================================
    # Define frequency bands
    config['freq_bands'] = {
                            'Theta': (3, 8),
                            'Alpha': (8, 13),
                            'Beta': (13, 30),
                            'Gamma': (30, 40),
                            # 'Broadband': (3, 40)
                            }

    # Define individualized frequency range over main peaks in each freq band
    config['individualized_band_ranges'] = { 
                                            'Theta': (-2, 3),
                                            'Alpha': (-2, 3), # change to (-4,2)
                                            'Beta': (-8, 9),
                                            'Gamma': (-5, 5)
                                            }

    # least acceptable R squred of fitted models
    config['min_r_squared'] = 0.9 
 
    config['feature_categories'] = {
                                    "Offset":True,
                                    "Exponent":True,
                                    "Peak_Center":True,
                                    "Peak_Power":True,
                                    "Peak_Width":True,
                                    "Adjusted_Canonical_Relative_Power":True, 
                                    "Adjusted_Canonical_Absolute_Power":True,
                                    "Adjusted_Individualized_Relative_Power":True,
                                    "Adjusted_Individualized_Absolute_Power":True,
                                    "OriginalPSD_Canonical_Relative_Power":True, 
                                    "OriginalPSD_Canonical_Absolute_Power":True,
                                    "OriginalPSD_Individualized_Relative_Power":True,
                                    "OriginalPSD_Individualized_Absolute_Power":True,
                                    }
    
    config["fooof_res_save_path"] = None

    config["random_state"] = 42

    if path is not None:
        out_file = open(os.path.join(path, project + ".json"), "w") 
        json.dump(config, out_file, indent = 6) 
        out_file.close()

    return config 

In [5]:
project = "test_feature_ex"

project_dir = f'/home/{username}/Results/{project}/'

mainParallel_path = os.path.join(package_path, 'src', 'mainParallel.py')

features_dir = os.path.join(project_dir, 'Features')
features_log_path = os.path.join(features_dir, 'log')
features_temp_path = os.path.join(features_dir,'temp')
pics_dir = os.path.join(project_dir, "pics")

nm_processing_dir = os.path.join(project_dir, 'NM', 'Run_' + str(run))

job_configs = {'log_path':features_log_path, 'module':'mne', 'time':'1:00:00', 'memory':'20GB', 
                'partition':'normal', 'core':1, 'node':1, 'batch_file_name':'batch_job'}

if not os.path.isdir(features_log_path):
    os.makedirs(features_log_path)

if not os.path.isdir(features_temp_path):
    os.makedirs(features_temp_path)
    
if not os.path.isdir(nm_processing_dir):
    os.makedirs(nm_processing_dir)

if not os.path.isdir(pics_dir):
    os.makedirs(pics_dir)
    
configs = make_config(project, project_dir)

subjects = merge_datasets_with_glob(datasets)

# f-IDPs extraction

In [ ]:
# Parallel feature extraction  

# Running Jobs
start_time = submit_jobs(mainParallel_path, features_dir, subjects, 
                features_temp_path, job_configs=job_configs, config_file=os.path.join(project_dir, project+'.json'))
# Checking jobs
failed_jobs = check_jobs_status(username, start_time)

falied_subjects = {failed_job:subjects[failed_job] for failed_job in failed_jobs}

while len(failed_jobs)>0:
    # Re-running Jobs
    start_time = submit_jobs(mainParallel_path, features_dir, falied_subjects, 
                features_temp_path, job_configs=job_configs, config_file=os.path.join(project_dir, project+'.json'))
    # Checking jobs
    failed_jobs = check_jobs_status(username, start_time)

collect_results(features_dir, subjects, features_temp_path, file_name='all_features', clean=False)

# Train-Test split

In [19]:
### Data preparation for Normative Modeling
data_base_dirs = [values["base_dir"] for values in datasets.values()]
dataset_names = list(datasets.keys())
merged_data, data_patient = merge_fidp_demo(data_base_dirs, features_dir, dataset_names, include_patients=False, diagnosis="parkinson")

biomarker_num = hbr_data_split(merged_data, nm_processing_dir, drop_nans=True, batch_effects=['sex', 'site'], random_seed=random_seeds[run], train_split=0.5)

biomarker_names = list(merged_data.columns[3:])

# EDA

In [ ]:
site = 4
print("Dataset name:", list(datasets.keys())[site])
print("size: ", merged_data[merged_data.site==site].shape[0], "participants")
print("mean age: ", merged_data[merged_data.site==site]["age"].mean())
print("std age: ", merged_data[merged_data.site==site]["age"].std())
print("female num: ", merged_data[np.logical_and(merged_data.site==site, merged_data.sex==1)].shape[0], "participants")

# NM configuration

In [21]:
### Setting up NM configs

python_path = '/project/meganorm/Software/Miniconda3/envs/mne/bin/python' 

hbr_configs = {
                'homo_Gaussian_linear':{'model_type':'linear', 'likelihood':'Normal', 'linear_sigma':'False',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'homo_Gaussian_bspline':{'model_type':'bspline', 'likelihood':'Normal', 'linear_sigma':'False',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'homo_SHASH_linear':{'model_type':'linear', 'likelihood':'SHASHb', 'linear_sigma':'False',
                                    'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'homo_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'False',
                                    'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'}, 
                'hetero_Gaussian_linear':{'model_type':'linear', 'likelihood':'Normal', 'linear_sigma':'True',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'},
                'hetero_Gaussian_bspline':{'model_type':'bspline', 'likelihood':'Normal', 'linear_sigma':'True',
                                        'random_slope_mu':'False', 'linear_epsilon':'False', 'linear_delta':'False'},
                'hetero_SHASH_linear':{'model_type':'linear', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                    'random_slope_mu':'False', 'linear_epsilon':'True', 'linear_delta':'True'},
                'hetero_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                        'random_slope_mu':'False', 'linear_epsilon':'True', 'linear_delta':'True'},
            }

inscaler='None' 
outscaler='None' 
batch_size = 1
outputsuffix = '_estimate'

respfile = os.path.join(nm_processing_dir, 'y_train.pkl')
covfile = os.path.join(nm_processing_dir, 'x_train.pkl')

testrespfile_path = os.path.join(nm_processing_dir, 'y_test.pkl')
testcovfile_path = os.path.join(nm_processing_dir, 'x_test.pkl')

trbefile = os.path.join(nm_processing_dir, 'b_train.pkl')
tsbefile = os.path.join(nm_processing_dir, 'b_test.pkl')

memory = '2gb'
duration = '5:00:00'
cluster_spec = 'slurm'

# Running NM

In [22]:
#for method in hbr_configs.keys():
method = 'hetero_SHASH_bspline'
processing_dir = os.path.join(nm_processing_dir, method) + '/'
nm_log_path = os.path.join(processing_dir, 'log') + '/'

if not os.path.isdir(processing_dir):
    os.makedirs(processing_dir)
if not os.path.isdir(nm_log_path):
    os.makedirs(nm_log_path)

execute_nm(processing_dir, python_path,
            'NM', covfile, respfile, batch_size, memory, duration, alg='hbr', 
            log_path=nm_log_path, binary=True, testcovfile_path=testcovfile_path, 
            testrespfile_path=testrespfile_path,trbefile=trbefile, tsbefile=tsbefile, 
            model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
            linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
            linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
            savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix=outputsuffix, 
            interactive='auto', cluster_spec=cluster_spec)

In [23]:
# collect_nm(processing_dir, "NM", collect=True, binary=True, batch_size=1)

# Model Diagnostic

In [ ]:
metrics_values = aggregate_metrics_across_runs(nm_processing_dir, method, biomarker_names, testcovfile_path, 
                              testrespfile_path, tsbefile,)
metrics_summary_path = os.path.join(processing_dir, "metrics_summary.pkl")
with open(metrics_summary_path, "wb") as file:
    pickle.dump(metrics_values, file)

plot_metrics(metrics_summary_path, 
            biomarker_names, 
            feature_new_name=["Theta", "Alpha", 
                            "Beta", "Gamma"], 
            save_path=pics_dir)

# Model comparison

In [ ]:
### Evaluating quantiles using MACE

mace, best_models, bio_ids = model_quantile_evaluation(hbr_configs, nm_processing_dir, testcovfile_path, 
                              testrespfile_path, tsbefile, biomarker_num, plot=False, outputsuffix='estimate')

plot_comparison(nm_processing_dir, hbr_configs, biomarker_num)

# NM ranges with markers

In [ ]:
# Plotting ranges
# # for config in hbr_configs.keys():
processing_path = os.path.join(nm_processing_dir, method)

q = estimate_centiles(processing_path, biomarker_num, quantiles=[0.05, 0.25, 0.5, 0.75, 0.95],
                        batch_map={0:{'Male':0, 'Female':1}, 1:{'BTNRH':0, 'CAMCAN':1, "NIMH":2, "OMEGA":3, "HCP":4}}, 
                        age_range=[6, 80])
plot_nm_range_site2(processing_path, nm_processing_dir)


# Age distribution (plot)

In [14]:
plot_age_dist2(merged_data, site_names=list(datasets.keys()), save_path=pics_dir)

# Test on clinical data

In [190]:
# random_seeds = [0]
for i in range(len(random_seeds)):

    nm_processing_dir_temp = nm_processing_dir.replace("Run_0", f"Run_{i}")
    processing_dir_temp = processing_dir.replace("Run_0", f"Run_{i}")

    prefix = "clinicalpredict_"
    prepare_prediction_data(data_patient.drop('diagnosis', axis=1),
                                nm_processing_dir_temp, 
                                drop_nans=True, 
                                batch_effects=['sex', 'site'], 
                                prefix=prefix)

    testrespfile_path = os.path.join(nm_processing_dir_temp, prefix + 'y_test.pkl')
    testcovfile_path = os.path.join(nm_processing_dir_temp, prefix + 'x_test.pkl')
    tsbefile = os.path.join(nm_processing_dir_temp, prefix + 'b_test.pkl')

    execute_nm(processing_dir_temp, python_path,
            'NM', testcovfile_path, testrespfile_path, batch_size, memory, duration, alg='hbr', 
            log_path=nm_log_path, binary=True, tsbefile=tsbefile, func="predict", 
            model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
            linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
            linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
            savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix="clinicalpredict", inputsuffix=outputsuffix,
            interactive='auto', cluster_spec=cluster_spec, nuts_sampler="nutpie", n_cores_per_batch="2")

# Abnormal probability index

In [ ]:
site_id = 3
p_vals, aucs = [], []

for i in range(len(random_seeds)):

    nm_processing_dir_temp = nm_processing_dir.replace("Run_0", f"Run_{i}")
    processing_dir_temp = processing_dir.replace("Run_0", f"Run_{i}")

    p_val, auc = abnormal_probability(processing_dir_temp,
                                    nm_processing_dir_temp, 
                                    site_id,
                                    n_permutation=1000)
    
    p_vals.append(p_val); aucs.append(auc)

p_vals = pd.DataFrame(np.vstack(p_vals))
aucs = pd.DataFrame(np.vstack(aucs))

aucs.columns = ["Theta", "Alpha", "Beta", "Gamma", "averaged"]

## AUC box plot

In [ ]:
box_plot_auc(aucs, save_path=pics_dir)

In [ ]:
aucs.mean()

# Scatter plot of Z-scores for patients with parkinson

In [ ]:
with open(os.path.join(processing_dir, "Z_clinicalpredict.pkl"), "rb") as file:
    z_patient = pickle.load(file)

z_patient.index = data_patient.index
z_patient.columns = biomarker_names


z_scores_scatter_plot(X = list(z_patient.loc[:, "Adjusted_Canonical_Relative_PowerTheta_all"]),
                    Y = list(z_patient.loc[:, "Adjusted_Canonical_Relative_PowerBeta_all"]),
                    bands_name=["theta", "beta"], 
                    thr=0.68,
                    save_path=pics_dir)

# INOCs

In [21]:
q_path = f"/home/{username}/Results/{project}/NM/Run_0/hetero_SHASH_bspline/Quantiles_estimate.pkl"

sub_index = "sub-042"
statistics = cal_stats_for_gauge(q_path, biomarker_names, 
                                 site_id=merged_data.loc[sub_index]["site"], 
                                 gender_id=merged_data.loc[sub_index]["sex"], 
                                 age=merged_data.loc[sub_index]["age"]*100)

new_names = ["Theta", "Alpha", "Beta", "Gamma"]

for i, name in enumerate(biomarker_names):

    if new_names[i] == "Gamma": max_value=0.2
    else: max_value=1

    plot_quantile_gauge(merged_data.loc[sub_index, name],
                        statistics[name][1],
                        statistics[name][3],
                        statistics[name][0],
                        statistics[name][4],
                        statistics[name][2],
                        title="",
                        max_value=max_value,
                        show_legend=False,
                        bio_name=new_names[i],
                        save_path=pics_dir
                        )
    
# sub-PD1674
# sub-PD1551
# sub-PD1517
# sub-MNI0079
# sub-PD0512
# sub-PD1487

# PNOCs

In [ ]:
# # Calculateing Oscilograms

gender_ids = {'Male':0, 'Female':1}
frequency_band_model_ids = {'Theta':0, 'Alpha':1, 'Beta':2, 'Gamma':3}
quantiles_path= os.path.join(processing_dir, 'Quantiles_estimate.pkl')
oscilograms, age_slices = calculate_oscilochart(quantiles_path, gender_ids, frequency_band_model_ids)

plot_neurooscillochart(oscilograms, age_slices, save_path=pics_dir)

# Plot growthchart using all of data

In [ ]:
plot_growthcharts(processing_dir, 
                  idp_indices=list(range(len(new_names))), 
                  idp_names = new_names)